In [16]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

In [17]:
load_dotenv()

True

In [18]:
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT']="LLM Structured Output Using Pydantic"
os.environ['LANGCHAIN_TRACING_V2']="true"

In [19]:
model=init_chat_model("ollama:llama3.1:8b")

In [20]:
model.invoke("hello?")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-02-16T10:31:22.692575569Z', 'done': True, 'done_reason': 'stop', 'total_duration': 260962793, 'load_duration': 85716193, 'prompt_eval_count': 12, 'prompt_eval_duration': 17286476, 'eval_count': 10, 'eval_duration': 150866843, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019c6601-3a3e-7530-acd9-a0cd37df1d23-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [21]:
from pydantic import BaseModel,Field
class Movie(BaseModel):
        title:str=Field(description="The title of the movie")
        year:int=Field(description="The year the movie was released")
        director:str=Field(description="The director of the movie")
        rating:float=Field(description="The movie rating out of 10  ")

In [23]:
model_with_structure=model.with_structured_output(Movie)

In [25]:
response=model_with_structure.invoke("Provide me the details of Avatar movie")

In [26]:
response

Movie(title='Avatar', year=2009, director='James Cameron', rating=8.3)

In [28]:
response=model_with_structure.invoke("Provide me the details of RRR movie")

In [29]:
response

Movie(title='RRR', year=2022, director='S.S. Rajamouli', rating=8.6)

### message output alongside structure  

In [31]:
model_with_structure=model.with_structured_output(Movie,include_raw=True)

In [32]:
response=model_with_structure.invoke("Provide me the details of RRR movie")

In [33]:
response

{'raw': AIMessage(content='{ "title": "RRR", "year": 2022, "director": "S. S. Rajamouli", "rating": 8.1 }\n\n    \t', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-02-16T10:36:29.611323196Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1236176869, 'load_duration': 86467960, 'prompt_eval_count': 18, 'prompt_eval_duration': 22995838, 'eval_count': 40, 'eval_duration': 646728192, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019c6605-e556-7161-b3f6-6825e72e283e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 18, 'output_tokens': 40, 'total_tokens': 58}),
 'parsed': Movie(title='RRR', year=2022, director='S. S. Rajamouli', rating=8.1),
 'parsing_error': None}

### nested structure

In [34]:
class Actor(BaseModel):
    name:str=Field(description="Actor name")
    role:str=Field(description="role")

class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float|None=Field(None,description="Budget in millions USD")

In [35]:
model_with_structure=model.with_structured_output(MovieDetails)
response=model_with_structure.invoke("Provide me the details of RRR movie")
response

MovieDetails(title='RRR', year=2022, cast=[Actor(name='N.T. Rama Rao Jr.', role='Komaram Bheem'), Actor(name='Ram Charan', role='Alluri Sitarama Raju'), Actor(name='Alia Bhatt', role='Sita'), Actor(name='Ajay Devgn', role='Bhavani Iyer'), Actor(name='Olivia Morris', role='Jennifer')], genres=['Action', 'Biographical Drama', 'Drama'], budget=550000000.0)